In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
sns.set()

In [49]:
df = pd.read_csv('Big market sales data.csv')
df.loc[df[df['Outlet_Identifier'] == 'OUT045'].index, 'Outlet_Size'] = 'High'
df.loc[df[df['Outlet_Identifier'] == 'OUT017'].index, 'Outlet_Size'] = 'High'
df.loc[df[df['Outlet_Identifier'] == 'OUT010'].index, 'Outlet_Size'] = 'High'
df['Item_Fat_Content'].replace(['LF', 'low fat'], 'Low Fat', inplace = True)
df['Item_Fat_Content'].replace('reg', 'Regular', inplace = True)
df.loc[df[df['Item_Weight'].isna()].index, 'Item_Weight'] = df['Item_Weight'].mean()

In [46]:
# df['Item_Fat_Content'] = df['Item_Fat_Content'].map((df.groupby(['Item_Fat_Content'])['Item_Outlet_Sales'].sum()/df.groupby(['Item_Fat_Content'])['Item_Outlet_Sales'].count()).to_dict())
df['Outlet_Type'] = df['Outlet_Type'].map((df.groupby(['Outlet_Type'])['Item_Outlet_Sales'].sum()/df.groupby(['Outlet_Type'])['Item_Outlet_Sales'].count()).to_dict())
# df['Outlet_Size'] = df['Outlet_Size'].map((df.groupby(['Outlet_Size'])['Item_Outlet_Sales'].sum()/df.groupby(['Outlet_Size'])['Item_Outlet_Sales'].count()).to_dict())
df['Outlet_Location_Type'] = df['Outlet_Location_Type'].map((df.groupby(['Outlet_Location_Type'])['Item_Outlet_Sales'].sum()/df.groupby(['Outlet_Location_Type'])['Item_Outlet_Sales'].count()).to_dict())
df['Item_Type'] = df['Item_Type'].map((df.groupby(['Item_Type'])['Item_Outlet_Sales'].sum()/df.groupby(['Item_Type'])['Item_Outlet_Sales'].count()).to_dict())
df['Outlet_Identifier_encoded'] = df['Outlet_Identifier'].map((df.groupby(['Outlet_Identifier'])['Item_Identifier'].count()/df['Item_Identifier'].nunique()).to_dict())
# df = pd.merge(df, df.groupby(['Outlet_Identifier', 'Item_Type'])['Item_Outlet_Sales'].mean().reset_index(), on = ['Outlet_Identifier', 'Item_Type'], suffixes=['', '_mean'])


In [47]:
df.corr()['Item_Outlet_Sales'].sort_values(ascending = False)

Item_Outlet_Sales            1.000000
Item_MRP                     0.567574
Outlet_Type                  0.489464
Outlet_Identifier_encoded    0.415103
Outlet_Location_Type         0.111828
Item_Type                    0.068843
Item_Weight                  0.011550
Outlet_Establishment_Year   -0.049135
Item_Visibility             -0.128625
Name: Item_Outlet_Sales, dtype: float64

In [ ]:
x = df[['Item_MRP', 'Outlet_Type', 'Outlet_Identifier_encoded','Item_Type','Item_Visibility', 'Outlet_Size']]
y = df['Item_Outlet_Sales']
x_train,x_test, y_train, y_test = train_test_split(x, y, random_state = 42, test_size = 0.2)

model = RandomForestRegressor(random_state = 42, criterion = 'mse', max_depth = 6, min_samples_leaf = 10,
                              max_features = 'auto', n_estimators = 80)

model.fit(x_train, y_train)
pred = model.predict(x_test)
print(model.score(x_train, y_train))
print(model.score(x_test, y_test))
print(model.feature_importances_)
print(np.sqrt(metrics.mean_squared_error(y_test, pred)))
# 0.6199273088429499
# 0.6195019028796055
# [0.57075229 0.17614554 0.18426108 0.01125856 0.00600452 0.051578  ]
# 1016.9476084724595


## Test Data

In [50]:
test_df = pd.read_csv('Big_market_test_data.csv')
test_df.loc[test_df[test_df['Outlet_Identifier'] == 'OUT045'].index, 'Outlet_Size'] = 'High'
test_df.loc[test_df[test_df['Outlet_Identifier'] == 'OUT017'].index, 'Outlet_Size'] = 'High'
test_df.loc[test_df[test_df['Outlet_Identifier'] == 'OUT010'].index, 'Outlet_Size'] = 'High'
test_df.loc[test_df[test_df['Item_Weight'].isna()].index, 'Item_Weight'] = test_df['Item_Weight'].mean()

test_df['Item_Fat_Content'].replace(['LF', 'low fat'], 'Low Fat', inplace = True)
test_df['Item_Fat_Content'].replace('reg', 'Regular', inplace = True)

test_df['Outlet_Type'] = test_df['Outlet_Type'].map((df.groupby(['Outlet_Type'])['Item_Outlet_Sales'].sum()/df.groupby(['Outlet_Type'])['Item_Outlet_Sales'].count()).to_dict())

test_df['Outlet_Location_Type'] = test_df['Outlet_Location_Type'].map((df.groupby(['Outlet_Location_Type'])['Item_Outlet_Sales'].sum()/df.groupby(['Outlet_Location_Type'])['Item_Outlet_Sales'].count()).to_dict())
test_df['Item_Type'] = test_df['Item_Type'].map((df.groupby(['Item_Type'])['Item_Outlet_Sales'].sum()/df.groupby(['Item_Type'])['Item_Outlet_Sales'].count()).to_dict())
test_df['Outlet_Identifier_encoded'] = test_df['Outlet_Identifier'].map((test_df.groupby(['Outlet_Identifier'])['Item_Identifier'].count()/test_df['Item_Identifier'].nunique()).to_dict())


In [51]:
test_df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Outlet_Identifier_encoded    0
dtype: int64

In [52]:
x_train.head(1)

,Item_MRP,Outlet_Type,Outlet_Identifier_encoded,Item_Type,Item_Visibility
549,171.3448,2316.181148,0.596536,2289.009592,0.035206


In [53]:
x = test_df[['Item_MRP', 'Outlet_Type','Outlet_Location_Type','Item_Type' ,'Outlet_Identifier_encoded']]
pred = model.predict(x)
pred

array([2102.92677462, 1831.32510518, 2539.41180362, ..., 2419.96394515,
       4438.58253081, 1825.93029964])

In [54]:
df_submit = test_df[['Item_Identifier', 'Outlet_Identifier']]
df_submit['Item_Outlet_Sales'] = pred

c:\users\anurag\appdata\local\programs\python\python35-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [55]:
df_submit.head(2)

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,2102.926775
1,FDW14,OUT017,1831.325105


In [56]:
df_submit.to_csv('Big data submit1.csv',index = False)